In [1]:
#

In [2]:
import string
from sklearn.feature_extraction.text import CountVectorizer
import nltk
from corus import load_taiga_fontanka_metas, load_taiga_fontanka

In [3]:
path = 'V:\TEMP_NLP\Fontanka.tar.gz'
#metas = load_taiga_fontanka_metas(path, offset=iTxt_offset, count=iTxt_count)
#records = load_taiga_fontanka(path, metas, offset=iTxt_offset, count=iTxt_count)
metas = load_taiga_fontanka_metas(path)
records = load_taiga_fontanka(path, metas)


In [4]:
iDataSetMax=342682 # из документации

In [5]:
dataset_src = [next(records) for i in range(iDataSetMax+1)]

In [6]:
#отбор данных за год вручную - из за ошибки в полях метаданных
dataset=[]
i=0
iBadData=0
while i in range (len(dataset_src)-1):    
    if dataset_src[i].meta.timestamp!='': # Ошибка в штампе времени
        if dataset_src[i].meta.timestamp.year == 2008:
            dataset.append(dataset_src[i])
    else:
        iBadData=i
        print('плохие данные ',iBadData)        
    i=i+1
    
print('полная число записей',len(dataset_src))    
print('число записей за один год',len(dataset))    
   


плохие данные  112775
полная число записей 342683
число записей за один год 26612


In [7]:
print('плохие метаданные')
dataset_src[iBadData].meta
#'20171009126' -последняя запись в датасете 
# рубрики - меняются, теги - константный массив

плохие метаданные


Meta(
    id='20110215020',
    timestamp='',
    tags='15.02.2011',
    themes=None,
    rubric='противоправных действий потерпевшего',
    genre=None,
    topic=None,
    author=None,
    lang=None,
    title='Адвокат «жемчужного ',
    url=None
)

In [8]:
dataset[-1].text


'В Гатчине успешно прошли публичные слушания по вопросу проекта планировки и межевания территории квартала №1 на въезде в город, где компания "Ленстройтрест" планирует построить новый, современный жилой район. В слушаниях приняли участие представители компании-застройщика, жители города и чиновники местной администрации.\nЭтот проект воплощает в себе все лучшее и современное, что есть в жилой архитектуре, а уникальное месторасположение – квартал начинается в створе улицы Чехова и Пушкинского шоссе и заканчивается у перекрестка трех дорог на въезде в Гатчину – обязывает его стать своеобразным лицом Гатчины, задавая тон новостройкам города. А потому к разработке градостроительной и архитектурной концепции нового квартала была привлечена ведущая шведская архитектурная мастерская TovattArchitects&Planners. При проектировании квартала не забыто и богатейшее культурно-историческое наследие города, бывшего любимой резиденцией императоров Павла I и Александра III. Объединить историю и современ

In [9]:

#nltk.download('punkt')
from nltk.tokenize import (
    sent_tokenize, 
    word_tokenize, 
    TweetTokenizer,
    WordPunctTokenizer, 
    WhitespaceTokenizer,
    LegalitySyllableTokenizer,
    SyllableTokenizer,
)
from nltk.stem.snowball import SnowballStemmer

In [10]:
russian_text=dataset[-1].text

words = nltk.word_tokenize(russian_text, language="russian")

# Удаляем из списка слов все знаки препинания и специальные символы
words = [word for word in words if word not in string.punctuation and not any(c.isdigit() for c in word)]

# Приводим слова к нижнему регистру
words = [word.lower() for word in words]

# Удаляем суффиксы при помощи алгоритма Портера
# Создаем экземпляр стеммера Портера для русского языка
stemmer = SnowballStemmer('russian')

# Удаляем знаки препинания и приводим слова к начальной форме при помощи стеммера
words = [stemmer.stem(word) for word in words if word.isalpha()]


In [11]:
clean_text_2 = " ".join(words)
clean_text_2

'в гатчин успешн прошл публичн слушан по вопрос проект планировк и межеван территор кварта на въезд в город где компан ленстройтрест планир постро нов современ жил район в слушан приня участ представител жител город и чиновник местн администрац этот проект воплоща в себ все лучш и современ что ест в жил архитектур а уникальн месторасположен кварта начина в створ улиц чехов и пушкинск шосс и заканчива у перекрестк трех дорог на въезд в гатчин обязыва ег стат своеобразн лиц гатчин задав тон новостройк город а пот к разработк градостроительн и архитектурн концепц нов кварта был привлеч ведущ шведск архитектурн мастерск tovattarchitects planners при проектирован кварта не забыт и богат наслед город бывш любим резиденц император павл i и александр iii объедин истор и современ позвол ландшафтн оформлен двор использован мал архитектурн форм и декоративн подсветк создан прогулочн зон изобил зелен при строительств нов кварта будут использова железобетон издел выпущен с помощ современ технологич

In [12]:
text=clean_text_2
# Очистка текста от стоп-слов и знаков препинания сделана выше

# Создание объекта CountVectorizer
vectorizer = CountVectorizer() #тексты заметок очень короткие - их надо склеивать либо под дате, либо по rubric

# Преобразование текста в матрицу счетчиков
matrix = vectorizer.fit_transform([text])

# Проверка, что матрица счетчиков была успешно создана
if matrix is not None:
    # Получение списка слов из словаря
    words = vectorizer.get_feature_names_out()
    
    # Вывод словаря
    print(words)
else:
    print("Ошибка: матрица счетчиков не была создана")

['iii' 'iv' 'planners' 'tovattarchitects' 'август' 'автолюбител'
 'администрац' 'александр' 'архитектур' 'архитектурн' 'банковск' 'близк'
 'богат' 'бронирован' 'будут' 'бывш' 'был' 'ведущ' 'весьм' 'вид' 'воплоща'
 'вопрос' 'все' 'встроен' 'въезд' 'выбра' 'выгляд' 'выпущен' 'высотн'
 'гатчин' 'гатчинск' 'гатчинц' 'где' 'глав' 'главн' 'год' 'город'
 'градостроительн' 'два' 'движен' 'двор' 'декоративн' 'детск' 'для'
 'должн' 'доминант' 'домостроен' 'дополнительн' 'дорог' 'доступн' 'ег'
 'ест' 'железобетон' 'жил' 'жител' 'забыт' 'заверш' 'заверя' 'задав'
 'задач' 'займут' 'заканчива' 'закрыт' 'запланирова' 'запуск' 'застройк'
 'застройщик' 'здан' 'зелен' 'знач' 'зон' 'из' 'издел' 'изобил'
 'император' 'интерес' 'использова' 'использован' 'истор' 'исход' 'их'
 'кажд' 'как' 'калугин' 'каф' 'кварта' 'квартир' 'климатическ' 'комбинат'
 'коммерческ' 'компан' 'конкурентоспособн' 'концепц' 'котор' 'ландшафтн'
 'ленстройтрест' 'лин' 'лиц' 'лучш' 'любим' 'магазин' 'мал' 'мастерск'
 'машин' 'машином